# Cell Counter
By Zoë McGinnis

## Loading in your image
The first thing we need to do is load in the pre-cropped image of the cell slide. Please put the image in the folder called "originals" and set `filename` to the name of the image you want to use. 

In [1]:
# CHANGE AS NEEDED TO SAY NAME OF THE DIRECTORY
folder = "originals"

## Install libraries (only need to run once)
Run this code block to install the libraries needed to run this program. You should only have to do this the very first time you use it.

In [2]:
%pip install matplotlib opencv-python Pillow


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import necessary libraries

In [2]:
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import csv

## Process all images in the folder

In [18]:
# Name the csv
csv_name = "{}.csv".format(folder)
# Open the file in the write mode
with open(folder + "/" + csv_name, 'w', newline='') as f:
    # Create the csv writer
    writer = csv.writer(f)
    # Write the header
    writer.writerow(["File name", "Num cells"])
    
    for filename in os.listdir(folder):
        # Skip the file if it's not an image
        if not filename.endswith(".png") and not filename.endswith(".jpg"):
            continue
        # Construct old file name
        source = folder + "/" + filename
        # Open image
        img = Image.open(source)
        width, height = img.size
    
        # Resize
        resized = img.resize((round(width/2), round(height/2)), Image.LANCZOS)
        new_img_name = folder + filename.split(".")[0] + "_small.png"
        # Save new image
        resized.save(new_img_name)

        # Open new image, creating multiple instances for plotting purposes
        im = cv2.imread(new_img_name)
        im_c = cv2.imread(new_img_name)
        img = cv2.imread(new_img_name, cv2.IMREAD_GRAYSCALE)
        
        # Remove image
        os.remove(new_img_name)

        # Otsu's thresholding after Gaussian filtering
        blur = cv2.GaussianBlur(img,(5,5),0)
        ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # Find and draw contours
        ret,thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        image = cv2.drawContours(im_c, contours, -1, (255,0,0), 1)
        num_cells = len(contours)
        
        # Write row to csv with the file name and number of cells counted
        writer.writerow([filename, num_cells])
        
# Close the csv
f.close()

/var/folders/px/z2sj3k8915xc72kmlt6jfhqc0000gn/T/ipykernel_31172/2904412768.py:21: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized = img.resize((round(width/2), round(height/2)), Image.LANCZOS)
